# Quadro geral para visualização no GeoSampa

Esse aqui é o NoteBook Para Planejamento da Visualização dos dados no GeoSampa


In [30]:
import pandas as pd
import glob
import os
# Abrindo o ano de 2020
df = pd.read_csv('download/dados-do-sp156---1o-sem-2015.csv',index_col=None, header=0, encoding="UTF-8", delimiter=',', parse_dates=['Data de abertura', 'Data do parecer'])

In [31]:
df_3tri = pd.read_csv('download/dados-do-sp156---3o-tri-2020.csv',index_col=None, header=0, encoding="UTF-8", delimiter=',', parse_dates=['Data de abertura', 'Data do parecer'])

In [32]:
df = pd.concat([df, df_3tri])
# df_3tri

In [2]:
df[df.Tema.str.len() < 5]['Tema'].index

Int64Index([   255,    353,   1807,   2293,   3940,   4631,   4773,   6570,
              7666,   7937,
            ...
            401972, 403810, 404321, 404414, 404651, 404798, 405676, 405701,
            405957, 406678],
           dtype='int64', length=1011)

In [5]:
df[df.Tema.str.len() < 5]['Tema'] = 'Outros'

<ipython-input-5-799f6049ae6b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df.Tema.str.len() < 5]['Tema'] = 'Outros'


In [9]:
df.loc[df.Tema.str.len() < 5, 'Tema'] = 'Outros'


In [10]:
df.loc[df.Tema.str.len() < 5, 'Tema']

Series([], Name: Tema, dtype: object)

In [14]:
df[df.Tema == 'Outro']

,Data de abertura,Canal,Tema,Assunto,Serviço,Logradouro,Número,CEP,Subprefeitura,Distrito,Setor,Quadra,Latitude,Longitude,Data do parecer,Status da solicitação,Órgão
255,2015-01-01 13:50:26.857,,Outro,1549,Apresentar sinais de embriaguez em serviço,NaN,NaN,NaN,ITAIM PAULISTA,VILA CURUCA,NaN,NaN,NaN,NaN,2019-08-26 10:05:32.251,FINALIZADA,SAC/GRC
353,2015-01-01 17:48:49.000,,Outro,2275,Localizacao de tumulo,NaN,NaN,NaN,VILA MARIA-VILA GUILHERME,VILA MEDEIROS,NaN,NaN,NaN,NaN,2018-04-13 10:13:20.000,FINALIZADA,SAC/GRC
1807,2015-01-02 14:18:22.860,,Outro,1472,Fio faiscando,NaN,NaN,NaN,SE,SE,NaN,NaN,NaN,NaN,2015-03-16 10:19:00.000,FINALIZADA,SAC/GRC
2293,2015-01-02 17:45:29.180,,Outro,1549,Apresentar sinais de embriaguez em serviço,NaN,NaN,NaN,CASA VERDE-CACHOEIRINHA,CACHOEIRINHA,NaN,NaN,NaN,NaN,2019-08-26 10:05:33.007,FINALIZADA,SAC/GRC
3940,2015-01-04 01:50:44.967,,Outro,1549,Apresentar sinais de embriaguez em serviço,NaN,NaN,NaN,VILA MARIANA,VILA MARIANA,NaN,NaN,NaN,NaN,2019-08-26 10:05:33.306,FINALIZADA,SAC/GRC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
404798,2015-06-29 11:47:26.317,,Outro,1687,Atualização de Dados Cadastrais,NaN,NaN,NaN,SANTO AMARO,CAMPO BELO,NaN,NaN,NaN,NaN,2019-03-07 15:15:19.000,FINALIZADA,SF
405676,2015-06-29 16:57:53.530,,Outro,2133,Sugestão,NaN,NaN,NaN,SE,SE,NaN,NaN,NaN,NaN,2019-03-07 14:48:24.907,FINALIZADA,SF
405701,2015-06-29 17:10:48.983,,Outro,1472,Fio faiscando,NaN,NaN,NaN,JABAQUARA,JABAQUARA,NaN,NaN,NaN,NaN,2015-07-22 09:07:00.000,FINALIZADA,SAC/GRC
405957,2015-06-29 19:44:44.000,,Outro,1392,Lavar/Reparar veiculo em via publica,NaN,NaN,NaN,CAMPO LIMPO,VILA ANDRADE,NaN,NaN,NaN,NaN,2018-04-13 10:13:20.000,FINALIZADA,SAC/GRC


In [4]:
df.reset_index(inplace=True)

In [8]:
df['Latitude']

0               NaN
1        -23.564410
2        -23.554494
3        -23.554640
4        -23.553785
            ...    
408242          NaN
408243          NaN
408244   -23.524180
408245          NaN
408246          NaN
Name: Latitude, Length: 408247, dtype: float64

In [9]:
# df['Latitude'] = pd.to_numeric(df['Latitude'].str.replace(',', '.'), errors='coerce')
# df['Longitude'] = pd.to_numeric(df['Longitude'].str.replace(',', '.'), errors='coerce')

In [10]:
df.dtypes

index                             int64
Data de abertura         datetime64[ns]
Canal                            object
Tema                             object
Assunto                          object
Serviço                          object
Logradouro                       object
Número                          float64
CEP                             float64
Subprefeitura                    object
Distrito                         object
Setor                           float64
Quadra                          float64
Latitude                        float64
Longitude                       float64
Data do parecer          datetime64[ns]
Status da solicitação            object
Órgão                            object
dtype: object

## Criando arquivo de LOG do processamento

Esse arquivo de log vai seguir documentando o processamento para servir de apêndice ao metadado

In [11]:
import os
if os.path.exists("./logs/log-visualizacao-sp156-2020.txt"):
    os.remove("./logs/log-visualizacao-sp156-2020.txt")

In [12]:
log = open("./logs/log-visualizacao-sp156-2020.txt", "w+")

In [13]:
from datetime import datetime
now = datetime.now()

In [14]:
log.write(f'## Iniciando o processamento da espacialização dos Atentimentos do SP 156\n')
log.write(f'- {datetime.now().strftime("%d/%m/%Y %H:%M:%S")}\n')

22

## Suprimindo da visualização serviços pouco solicitados 

Para melhorar a navagação seria interessante ter menos serviços dessa forma podemos criar um grupo grande com outros serviços

In [15]:
log.write('## Removendo serviços com menos de 200 ocorrências\n')

51

In [16]:
limite = 200

In [17]:
%pprint

Pretty printing has been turned OFF


In [18]:
log.write('- Documentando serviços que ficarão de fora da visualização\n')
log.write(f'- salvando o resultado em ./resultados/hierarquia-de-visualizacao-abaixo-limite-{limite}.csv\n')

88

In [19]:
df[~df.Distrito.isna()].groupby(['Serviço']).filter(lambda x: len(x) > limite).groupby(['Tema','Serviço'])['Canal'].count().to_csv(f'./resultados/hierarquia-de-visualizacao-limite-{limite}.csv')

In [20]:
log.write('- Documentando serviços com mais de 200 ocorrências visualização\n')
log.write(f'- salvando o resultado em ./resultados/hierarquia-de-visualizacao-limite-{limite}.csv\n')

81

In [21]:
df[~df.Distrito.isna()].groupby(['Serviço']).filter(lambda x: len(x) <= limite).groupby(['Tema','Serviço'])['Canal'].count().to_csv(f'./resultados/hierarquia-de-visualizacao-abaixo-limite-{limite}.csv')

In [22]:
df_menor_200 = df.drop(df.groupby(['Serviço']).filter(lambda x: len(x) > limite).index)

In [23]:
df = df.drop(df.groupby(['Serviço']).filter(lambda x: len(x) <= limite).index)

In [24]:
df

,index,Data de abertura,Canal,Tema,Assunto,Serviço,Logradouro,Número,CEP,Subprefeitura,Distrito,Setor,Quadra,Latitude,Longitude,Data do parecer,Status da solicitação,Órgão
0,0,2015-01-01 00:01:46.310,,Cidadania e assistência social,População ou pessoa em situação de rua,Serviço Especializado de Abordagem Social às P...,NaN,NaN,NaN,GUAIANASES,GUAIANASES,NaN,NaN,NaN,NaN,2015-01-01 06:00:54.713,FINALIZADA,SMADS
1,1,2015-01-01 00:08:31.000,,Rua e bairro,"Reparos em asfalto, pontes e viadutos",Tapa-buraco,RUA LICURI,30.0,3585090.0,ITAQUERA,CIDADE LIDER,146.0,161.0,-23.564410,-46.490931,2015-01-20 11:11:42.310,FINALIZADA,SMSUB
2,2,2015-01-01 00:09:24.093,,Trânsito e Transporte,Reclamação ônibus,Ônibus - Reclamação de intervalo excessivo da ...,RUA EMILIA MARENGO,801.0,3336000.0,MOOCA,TATUAPE,54.0,229.0,-23.554494,-46.559820,2015-02-09 16:17:00.000,FINALIZADA,SPTRANS
3,3,2015-01-01 00:12:24.000,,Rua e bairro,"Reparos em asfalto, pontes e viadutos",Tapa-buraco,RUA PEDRO DE LABATUT,10.0,8280020.0,ITAQUERA,CIDADE LIDER,146.0,139.0,-23.554640,-46.481812,2015-01-22 11:30:27.997,FINALIZADA,SMSUB
4,4,2015-01-01 00:13:48.000,,Rua e bairro,"Reparos em asfalto, pontes e viadutos",Tapa-buraco,RUA JOSE DORIA DE ANDRADE,28.0,8285340.0,ITAQUERA,CIDADE LIDER,145.0,9.0,-23.553785,-46.480517,2015-01-22 11:30:11.740,FINALIZADA,SMSUB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
408242,408242,2015-06-30 23:56:47.833,,Rua e bairro,Poluição Sonora - PSIU,Reclamação de poluição sonora - PSIU,NaN,NaN,NaN,CIDADE ADEMAR,PEDREIRA,NaN,NaN,NaN,NaN,2015-07-28 10:59:43.360,FINALIZADA,SMSUB
408243,408243,2015-06-30 23:56:56.000,,Trânsito e Transporte,Veículos abandonados,Remoção de veículo/carcaça abandonado na via p...,NaN,NaN,NaN,ITAQUERA,CIDADE LIDER,NaN,NaN,NaN,NaN,2018-07-16 18:13:09.000,FINALIZADA,SMSUB
408244,408244,2015-06-30 23:57:10.710,,Lixo e limpeza,Varrição e limpeza urbana,Limpeza da via pública após enchentes ou eventos,RUA HENRIQUE,784.0,3664010.0,PENHA,PENHA,59.0,138.0,-23.524180,-46.498181,2015-07-13 00:00:00.000,FINALIZADA,AMLURB
408245,408245,2015-06-30 23:59:26.000,,Trânsito e Transporte,Veículos abandonados,Remoção de veículo/carcaça abandonado na via p...,NaN,NaN,NaN,JACANA-TREMEMBE,TREMEMBE,NaN,NaN,NaN,NaN,2015-09-11 12:53:43.460,FINALIZADA,SMSUB


In [25]:
df[df.Serviço.str.contains('morcego')]

,index,Data de abertura,Canal,Tema,Assunto,Serviço,Logradouro,Número,CEP,Subprefeitura,Distrito,Setor,Quadra,Latitude,Longitude,Data do parecer,Status da solicitação,Órgão
170,170,2015-01-01 11:07:40.080,,Animais,Animais que podem causar doenças e/ou agravos ...,Morcego - Reclamar sobre local com morcego,RUA FERREIRA ALVES,NaN,NaN,LAPA,PERDIZES,22.0,51.0,NaN,NaN,2017-05-31 15:08:05.804,FINALIZADA,COVISA-DVZ
1811,1811,2015-01-02 14:22:02.183,,Animais,Animais que podem causar doenças e/ou agravos ...,Morcego - Reclamar sobre local com morcego,RUA SA DE MIRANDA,NaN,NaN,PINHEIROS,ITAIM BIBI,299.0,119.0,NaN,NaN,2017-05-31 15:19:23.181,FINALIZADA,COVISA-DVZ
3173,3173,2015-01-03 12:12:26.693,,Animais,Animais que podem causar doenças e/ou agravos ...,Morcego - Reclamar sobre local com morcego,RUA PEDRO POMPONAZZI,NaN,NaN,VILA MARIANA,VILA MARIANA,39.0,158.0,NaN,NaN,2018-04-12 09:05:29.627,FINALIZADA,COVISA-DVZ
4108,4108,2015-01-04 10:55:33.907,,Animais,Animais que podem causar doenças e/ou agravos ...,Morcego - Reclamar sobre local com morcego,RUA TIRANA,NaN,NaN,ARICANDUVA-FORMOSA-CARRAO,VILA FORMOSA,55.0,227.0,NaN,NaN,2018-04-12 09:05:29.241,FINALIZADA,COVISA-DVZ
5088,5088,2015-01-05 08:09:43.313,,Animais,Animais que podem causar doenças e/ou agravos ...,Morcego - Reclamar sobre local com morcego,RUA CANCIONEIRO DE EVORA,NaN,NaN,SANTO AMARO,SANTO AMARO,85.0,14.0,NaN,NaN,2017-05-31 15:21:24.567,FINALIZADA,COVISA-DVZ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
405213,405213,2015-06-29 14:15:36.207,,Animais,Animais que podem causar doenças e/ou agravos ...,Morcego - Reclamar sobre local com morcego,RUA TUTOIA,NaN,NaN,VILA MARIANA,VILA MARIANA,36.0,45.0,NaN,NaN,2017-06-07 14:19:01.043,FINALIZADA,COVISA-DVZ
405293,405293,2015-06-29 14:43:25.927,,Animais,Animais que podem causar doenças e/ou agravos ...,Morcego - Reclamar sobre local com morcego,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-06-07 14:21:59.533,FINALIZADA,COVISA-DVZ
405756,405756,2015-06-29 17:36:50.910,,Animais,Animais que podem causar doenças e/ou agravos ...,Morcego - Reclamar sobre local com morcego,RUA IGARAPE AGUA AZUL,NaN,NaN,CIDADE TIRADENTES,CIDADE TIRADENTES,246.0,12.0,NaN,NaN,2017-06-07 14:25:31.158,FINALIZADA,COVISA-DVZ
406401,406401,2015-06-30 08:35:32.403,,Animais,Animais que podem causar doenças e/ou agravos ...,Morcego - Reclamar sobre local com morcego,RUA LUIZ MIGLIANO,NaN,NaN,BUTANTA,VILA SONIA,171.0,258.0,NaN,NaN,2018-04-12 09:03:44.304,FINALIZADA,COVISA-DVZ


## Pontos

No caso dos pontos podemos gerar um arquivo sem discretização, ou seja, cada registro conterá seu ponto.

Então para todos os registros vamos criar uma geometria de pontos pela Lat e Long

In [26]:
import geopandas as gpd

In [27]:
log.write(f'## Convertendo Lat/Long (epsg:4989) em UTM SIRGAS 2000 (epsg:31983)\n')

68

In [28]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude))
gdf_menor_200 = gpd.GeoDataFrame(df_menor_200, geometry=gpd.points_from_xy(df_menor_200.Longitude, df_menor_200.Latitude))

In [29]:
gdf.crs = 'epsg:4989'
gdf = gdf.to_crs('epsg:31983')
gdf_menor_200.crs = 'epsg:4989'
gdf_menor_200 = gdf_menor_200.to_crs('epsg:31983')

In [30]:
gdf[gdf.geometry.is_valid]

,index,Data de abertura,Canal,Tema,Assunto,Serviço,Logradouro,Número,CEP,Subprefeitura,Distrito,Setor,Quadra,Latitude,Longitude,Data do parecer,Status da solicitação,Órgão,geometry
1,1,2015-01-01 00:08:31.000,,Rua e bairro,"Reparos em asfalto, pontes e viadutos",Tapa-buraco,RUA LICURI,30.0,3585090.0,ITAQUERA,CIDADE LIDER,146.0,161.0,-23.564410,-46.490931,2015-01-20 11:11:42.310,FINALIZADA,SMSUB,POINT (347838.028 7393206.409)
2,2,2015-01-01 00:09:24.093,,Trânsito e Transporte,Reclamação ônibus,Ônibus - Reclamação de intervalo excessivo da ...,RUA EMILIA MARENGO,801.0,3336000.0,MOOCA,TATUAPE,54.0,229.0,-23.554494,-46.559820,2015-02-09 16:17:00.000,FINALIZADA,SPTRANS,POINT (340794.224 7394229.627)
3,3,2015-01-01 00:12:24.000,,Rua e bairro,"Reparos em asfalto, pontes e viadutos",Tapa-buraco,RUA PEDRO DE LABATUT,10.0,8280020.0,ITAQUERA,CIDADE LIDER,146.0,139.0,-23.554640,-46.481812,2015-01-22 11:30:27.997,FINALIZADA,SMSUB,POINT (348757.666 7394297.904)
4,4,2015-01-01 00:13:48.000,,Rua e bairro,"Reparos em asfalto, pontes e viadutos",Tapa-buraco,RUA JOSE DORIA DE ANDRADE,28.0,8285340.0,ITAQUERA,CIDADE LIDER,145.0,9.0,-23.553785,-46.480517,2015-01-22 11:30:11.740,FINALIZADA,SMSUB,POINT (348888.843 7394393.913)
5,5,2015-01-01 00:14:53.000,,Rua e bairro,"Reparos em asfalto, pontes e viadutos",Tapa-buraco,RUA VITO MODESTO PEDOTE,50.0,3570190.0,ITAQUERA,CIDADE LIDER,147.0,265.0,-23.561564,-46.488176,2015-01-20 11:11:24.797,FINALIZADA,SMSUB,POINT (348115.959 7393524.490)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
408226,408226,2015-06-30 23:18:46.800,,Trânsito e Transporte,Reclamação ônibus,Ônibus - Denúncia de problemas no embarque/des...,AVENIDA JOSE DINIZ,0.0,4603004.0,SANTO AMARO,SANTO AMARO,88.0,44.0,-23.638529,-46.695491,2015-07-20 10:03:00.000,FINALIZADA,SPTRANS,POINT (327054.288 7384766.161)
408230,408230,2015-06-30 23:23:58.423,,Trânsito e Transporte,Reclamação ônibus,Ônibus - Reclamação de descumprimento de parti...,AVENIDA FLAMINGO,0.0,8031000.0,ITAIM PAULISTA,VILA CURUCA,139.0,392.0,-23.509231,-46.422615,2015-07-15 09:24:00.000,FINALIZADA,SPTRANS,POINT (354750.628 7399387.335)
408233,408233,2015-06-30 23:27:10.000,,Rua e bairro,Drenagem de água de chuva,(antigo) Reformar galerias de águas de chuvas ...,RUA GONCALO AFONSO,64.0,5436100.0,PINHEIROS,PINHEIROS,81.0,176.0,-23.556708,-46.685929,2015-07-03 11:20:51.193,FINALIZADA,SMSUB,POINT (327923.035 7393838.597)
408234,408234,2015-06-30 23:32:11.000,,Rua e bairro,Drenagem de água de chuva,(antigo) Reformar galerias de águas de chuvas ...,AVENIDA GALERA,0.0,3725110.0,PENHA,CANGAIBA,110.0,39.0,-23.497481,-46.516885,2016-12-15 16:52:28.693,INDEFERIDO,SMSUB,POINT (345110.440 7400589.917)


## Tratando pontos fora dos limites do município de São Paulo

Existem casos em que os pontos estão localizados fora dos limites do município de São Paulo e para finalidade de estudo vamos quantifica-los e discriminá-los para estudos posteriores.

In [31]:
msp = gpd.read_file('arquivos/SP.shp')
gdf_sp = gpd.sjoin(gdf, msp, how='left', op='within')

In [32]:
gdf[gdf_sp.municipio.isna()]

,index,Data de abertura,Canal,Tema,Assunto,Serviço,Logradouro,Número,CEP,Subprefeitura,Distrito,Setor,Quadra,Latitude,Longitude,Data do parecer,Status da solicitação,Órgão,geometry
0,0,2015-01-01 00:01:46.310,,Cidadania e assistência social,População ou pessoa em situação de rua,Serviço Especializado de Abordagem Social às P...,NaN,NaN,NaN,GUAIANASES,GUAIANASES,NaN,NaN,NaN,NaN,2015-01-01 06:00:54.713,FINALIZADA,SMADS,POINT (inf inf)
8,8,2015-01-01 00:19:20.083,,Cidadania e assistência social,População ou pessoa em situação de rua,Serviço Especializado de Abordagem Social às P...,NaN,NaN,NaN,SE,SANTA CECILIA,NaN,NaN,NaN,NaN,2015-01-01 00:59:43.890,FINALIZADA,SMADS,POINT (inf inf)
38,38,2015-01-01 01:06:00.000,,Rua e bairro,Drenagem de água de chuva,Drenagem de água da chuva - Bueiros e bocas de...,NaN,NaN,NaN,ITAQUERA,CIDADE LIDER,NaN,NaN,NaN,NaN,2015-01-09 11:16:01.547,FINALIZADA,SMSUB,POINT (inf inf)
39,39,2015-01-01 01:06:42.000,,Rua e bairro,Drenagem de água de chuva,Drenagem de água da chuva - Bueiros e bocas de...,NaN,NaN,NaN,PENHA,VILA MATILDE,NaN,NaN,NaN,NaN,2015-01-19 00:00:00.000,FINALIZADA,SMSUB,POINT (inf inf)
40,40,2015-01-01 01:07:13.000,,Rua e bairro,Iluminação pública,Reparo da fiação elétrica,NaN,NaN,NaN,IPIRANGA,CURSINO,NaN,NaN,NaN,NaN,2018-04-12 08:32:21.114,FINALIZADA,ILUME,POINT (inf inf)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
408241,408241,2015-06-30 23:55:20.460,,Rua e bairro,Poluição Sonora - PSIU,Reclamação de poluição sonora - PSIU,NaN,NaN,NaN,SE,BELA VISTA,NaN,NaN,NaN,NaN,2015-07-06 12:28:41.417,FINALIZADA,SMSUB,POINT (inf inf)
408242,408242,2015-06-30 23:56:47.833,,Rua e bairro,Poluição Sonora - PSIU,Reclamação de poluição sonora - PSIU,NaN,NaN,NaN,CIDADE ADEMAR,PEDREIRA,NaN,NaN,NaN,NaN,2015-07-28 10:59:43.360,FINALIZADA,SMSUB,POINT (inf inf)
408243,408243,2015-06-30 23:56:56.000,,Trânsito e Transporte,Veículos abandonados,Remoção de veículo/carcaça abandonado na via p...,NaN,NaN,NaN,ITAQUERA,CIDADE LIDER,NaN,NaN,NaN,NaN,2018-07-16 18:13:09.000,FINALIZADA,SMSUB,POINT (inf inf)
408245,408245,2015-06-30 23:59:26.000,,Trânsito e Transporte,Veículos abandonados,Remoção de veículo/carcaça abandonado na via p...,NaN,NaN,NaN,JACANA-TREMEMBE,TREMEMBE,NaN,NaN,NaN,NaN,2015-09-11 12:53:43.460,FINALIZADA,SMSUB,POINT (inf inf)


## Transformando Setor e Quadra em pontos

Alguns registros de atendimento estão com informações de quadras ou setores fiscais e portanto deveriam ser representado pela feição do polígono

Porém, para evitarmos múltiplas feições ou uma subtotalização transformar aqueles registros em um ponto no centroide da feição, uma vez que conhecemos as feições de Setor e Quadra. Como são casos menos frequentes podemos proporcionar uma melhor análise dessa maneira.

Alias, poderíamos criar um ponto aleatório na feição para evitar a sobreposição, o que também pode ser uma estratégia.

In [33]:
log.write('## Transformando Registros que contém Setor e Quadra em pontos de Centroide das respectivas feições\n')

100

In [34]:
zipfile = "zip:///home/fernando/dev/geo-sp156/arquivos/SIRGAS_SHP_setorfiscal.zip!SIRGAS_SHP_setorfiscal/SIRGAS_SHP_setorfiscal.shp"
df_setores = gpd.read_file(zipfile)

zipfile = "zip:///home/fernando/dev/geo-sp156/arquivos/SIRGAS_SHP_quadraMDSF.zip!SIRGAS_SHP_quadraMDSF/SIRGAS_SHP_quadraMDSF.shp"
df_quadras = gpd.read_file(zipfile)

In [35]:
df_quadras = df_quadras[df_quadras.qd_tipo == 'F']

In [36]:
df_setor = gdf[gdf.Latitude.isna() & ~gdf.Setor.isna() & gdf.Quadra.isna()]
df_quadra = gdf[gdf.Latitude.isna() & ~gdf.Setor.isna() & ~gdf.Quadra.isna()]

In [37]:
df_setor['st_codigo'] = df_setor.Setor.astype(int).apply(lambda x: '{:03d}'.format(x))
df_quadra['qd_setor'] = df_quadra.Setor.astype(int).apply(lambda x: '{:03d}'.format(x))
df_quadra['qd_fiscal'] = df_quadra.Quadra.astype(int).apply(lambda x: '{:03d}'.format(x))

/home/fernando/miniconda3/envs/geopandas/lib/python3.8/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [38]:
df_setor.st_codigo = df_setor.st_codigo.astype('str')
df_quadra.qd_setor = df_quadra.qd_setor.astype('str')
df_quadra.qd_fiscal = df_quadra.qd_fiscal.astype('str')

In [39]:
# REmovendo setores duplicados
df_setores = df_setores.drop(df_setores[df_setores.duplicated(['st_codigo'], keep=False)].index)

In [40]:
df_setores.geometry = df_setores.geometry.centroid

In [41]:
df_setor['id'] = df_setor.index

In [42]:
setor = df_setor.merge(df_setores[['geometry', 'st_codigo']], on=['st_codigo'], suffixes=('', '_y'), how='left')

In [43]:
setor = setor.set_index('id')

In [44]:
# Atualizando os centroind no DataFrame
gdf.loc[(gdf.Latitude.isna() & ~gdf.Setor.isna() & gdf.Quadra.isna()), 'geometry'] = setor.geometry_y

In [45]:
# REmovendo as quadras duplicadas
quadras = df_quadras.drop(df_quadras[df_quadras.duplicated(['qd_setor', 'qd_fiscal', 'qd_subqua'], keep=False)].index)

In [46]:
quadras = quadras.drop(quadras[quadras.qd_subqua != '001'].index)

In [47]:
quadras.geometry = quadras.geometry.centroid

In [48]:
df_quadra['id'] = df_quadra.index

In [49]:
quadra = df_quadra.merge(quadras[['geometry', 'qd_setor', 'qd_fiscal']], on=['qd_setor', 'qd_fiscal'], suffixes=('', '_y'), how='left')

In [50]:
quadra = quadra.set_index('id')

In [51]:
# Atualizando os centroind da quadra no DataFrame
gdf.loc[gdf.Latitude.isna() & ~gdf.Setor.isna() & ~gdf.Quadra.isna(), 'geometry'] = quadra.geometry_y

In [52]:
# gdf[gdf.geometry.is_valid].to_file('resultados/atendimentos-1-semestre-2020.gpkg', driver='GPKG')

In [53]:
# gdf[gdf.Serviço.str.contains('morcego')].is_valid.value_counts()

In [54]:
gdf.Serviço.unique()

array(['Serviço Especializado de Abordagem Social às Pessoas em Situação de Rua - SEAS',
       'Tapa-buraco',
       'Ônibus - Reclamação de intervalo excessivo da linha',
       'Ônibus - Reclamação de descumprimento de partida no ponto inicial/final',
       'Drenagem de água da chuva - Bueiros e bocas de lobo',
       'Reparo da fiação elétrica', 'Reparo de lâmpada',
       'SPTrans - Sugestão',
       'Avaliação de animal em sofrimento sem proprietário para eutanásia (morte sem dor)',
       'Calçada pública - solicitar manutenção',
       'Acessibilidade - solicitar avaliação de obstáculo na calçada',
       'Árvore - solicitar manejo em área pública',
       'Reclamação de problemas no olho vivo',
       'Reclamação de poluição sonora - PSIU',
       'Remoção de veículo/carcaça abandonado na via pública',
       'Ônibus - Denúncia de problemas no embarque/desembarque',
       'Comércio, serviço e indústria - solicitar fiscalização de funcionamento',
       'Reclamações sobre col

## Totalizando serviços por distrito

Quando o atendimento gera apenas uma informação do distrito uma estratégia é totalizar os atendimentos por cada distrito.

In [55]:
log.write('## Totalizando serviços por Distrito\n')

37

In [56]:
df_distritos = gpd.read_file('arquivos/SIRGAS_SHP_distrito_polygon.shp')

In [57]:
df_distritos.crs = 'epsg:31983'

In [58]:
log.write(f'- Removendo {len(df[df.Distrito.isna()])} registros que não tem informação de distrito nem outra informação espacial\n')

93

In [59]:
df_distrito = df[~df.Distrito.isna()]

In [60]:
log.write(f'- Totalizando {len(df_distrito.Tema.unique())} temas distintos\n')

33

In [61]:
log.write(f'- Totalizando {len(df_distrito.Serviço.unique())} serviços distintos\n')

37

In [62]:
# pd.get_dummies(df[['Distrito', 'Serviço']][~df['Distrito'].isna()], columns=['Serviço']).groupby(['Distrito']).sum()

In [63]:
log.write(f'- Totalizando {len(gdf[~gdf.geometry.is_valid & ~gdf.Quadra.isna() & ~gdf.Setor.isna()])} registros sem Setor e Quadra válidos\n')

54

In [64]:
log.write("## Gerando totalizações de camadas\n")

35

In [65]:
# for t in gdf.Tema.unique():
#     print(f"{t}: ({len(gdf[(gdf.Tema == t)])})")
#     log.write(f"{t}: ({len(gdf[(gdf.Tema == t)])})\n")
#     for s in gdf[(gdf.Tema == t)].Serviço.unique():
#         g = len(gdf[(gdf.Serviço == s)])
#         g_valids = len(gdf[gdf.geometry.is_valid & (gdf.Serviço == s)])
#         print(f"  {s}: ({g}: {g_valids} pontos, {g-g_valids} agregado por distrito)")
#         log.write(f"  {s}: ({g}: {g_valids} pontos, {g-g_valids} agregado por distrito)\n")

## Centroide do Distrito para casos em que alguns registros não são localizados

Alguns serviços possuem uma  baixa porcentagem de casos onde não há possibilidade de localização. Para isso vamos determinar que até 20% de registros não localizados, serão referenciados no centroide do distrito afim de continua-la tratando como camada de pontos

In [66]:
df_distritos

,ds_codigo,ds_nome,geometry
0,51,MANDAQUI,"POLYGON ((330950.373 7407837.176, 330952.177 7..."
1,52,MARSILAC,"POLYGON ((336124.090 7355302.282, 336121.755 7..."
2,32,MOEMA,"POLYGON ((331242.174 7392162.324, 331244.581 7..."
3,57,PARQUE DO CARMO,"POLYGON ((352597.619 7390119.444, 352588.233 7..."
4,60,PERDIZES,"POLYGON ((330181.745 7396056.840, 330175.535 7..."
...,...,...,...
91,41,JAGUARE,"POLYGON ((322952.516 7394978.618, 323250.620 7..."
92,42,JARAGUA,"POLYGON ((319172.403 7405179.204, 319174.204 7..."
93,44,JARDIM HELENA,"POLYGON ((352915.621 7402621.049, 352944.248 7..."
94,45,JARDIM PAULISTA,"POLYGON ((329605.461 7390837.489, 329600.764 7..."


In [67]:
gdf_servicos_pontos = gdf[gdf.is_valid].groupby(by=['Serviço']).agg({
    'geometry':'count'}).rename(columns={'geometry':'total_atendimentos_pontos'})

In [68]:
diferenca_total_distritos = pd.concat([gdf[gdf.Serviço.isin(gdf_servicos_pontos.index)].groupby(by=['Serviço']).agg({
    'geometry':'count'}).rename(columns={'geometry':'total_atendimentos'}),gdf_servicos_pontos], axis=1)

In [69]:
diferenca_total_distritos[(diferenca_total_distritos.total_atendimentos_pontos/diferenca_total_distritos.total_atendimentos)>0.8]

,total_atendimentos,total_atendimentos_pontos
Serviço,,
(antigo) Reformar galerias de águas de chuvas (pluviais),940,940
Bilhete Único - Falha na recarga,390,390
Cata Bagulho - Reclamação de não retirada de material,1573,1573
Comunicação de erro em cobrança de integração (ônibus e metrô ou trem),2021,2021
Comunicação de problema em posto de recarga,1228,1228
Criação/reativação de linha de ônibus,344,344
Implantação de abrigo ou ponto de ônibus,1571,1571
Instalação de contêiner para coleta de lixo domiciliar,394,394
Instalação e remoção de lixeira,767,767


In [70]:
gdf_servicos_pontos = diferenca_total_distritos[(diferenca_total_distritos.total_atendimentos_pontos/diferenca_total_distritos.total_atendimentos)>0.8]

In [71]:
servicos_sem_ponto_com_distrito = gdf[gdf.Serviço.isin(gdf_servicos_pontos.index) & ~gdf.is_valid & ~gdf.Subprefeitura.isna()]['Distrito']

In [72]:
servicos_sem_ponto_com_distrito

912              ARICANDUVA
2997                LAJEADO
3629          CIDADE ADEMAR
3672          JARDIM ANGELA
3830          CIDADE ADEMAR
                ...        
404327          CAMPO LIMPO
405756    CIDADE TIRADENTES
406545          VILA CURUCA
406684         CIDADE DUTRA
407681             CANGAIBA
Name: Distrito, Length: 470, dtype: object

In [73]:
servicos_sem_ponto_com_distrito = pd.DataFrame(servicos_sem_ponto_com_distrito)
servicos_sem_ponto_com_distrito['index'] = servicos_sem_ponto_com_distrito.index

In [74]:
servicos_sem_ponto_com_distrito

,Distrito,index
912,ARICANDUVA,912
2997,LAJEADO,2997
3629,CIDADE ADEMAR,3629
3672,JARDIM ANGELA,3672
3830,CIDADE ADEMAR,3830
...,...,...
404327,CAMPO LIMPO,404327
405756,CIDADE TIRADENTES,405756
406545,VILA CURUCA,406545
406684,CIDADE DUTRA,406684


In [75]:
df_distritos['centroide'] = df_distritos.geometry.centroid

In [76]:
centroide_distritos = servicos_sem_ponto_com_distrito.merge(df_distritos, left_on='Distrito', right_on='ds_nome').set_index('index').centroide

In [73]:
gdf.is_valid.value_counts()

False    794031
True     207937
dtype: int64

In [74]:
#### TODO não está atribuindo o centroide

# gdf
# gdf.iloc[gdf.index.isin(centroide_distritos.index)].geometry #= centroide_distritos
centroide_distritos.index.name = ''
centroide_distritos


363        POINT (319690.025 7376055.851)
393        POINT (319690.025 7376055.851)
1192       POINT (319690.025 7376055.851)
9968       POINT (319690.025 7376055.851)
11151      POINT (319690.025 7376055.851)
                        ...              
944967     POINT (333445.768 7392767.705)
944974     POINT (333445.768 7392767.705)
982953     POINT (333445.768 7392767.705)
1002090    POINT (333445.768 7392767.705)
1017769    POINT (333445.768 7392767.705)
Name: centroide, Length: 8828, dtype: geometry

In [75]:
gdf.iloc[gdf.index.isin(centroide_distritos.index), gdf.columns.get_loc('geometry')] = centroide_distritos

In [76]:
gdf.iloc[gdf.index.isin(centroide_distritos.index), gdf.columns.get_loc('geometry')]

363        POINT (319690.025 7376055.851)
393        POINT (319690.025 7376055.851)
440        POINT (331801.371 7377266.082)
485        POINT (331045.774 7380907.159)
590        POINT (329981.185 7368063.452)
                        ...              
1019061    POINT (345985.963 7388629.437)
1019122    POINT (325151.304 7363541.607)
1019281    POINT (327531.212 7405808.370)
1019310    POINT (343888.326 7397736.879)
1019331    POINT (343888.326 7397736.879)
Name: geometry, Length: 8828, dtype: geometry

## Gerando o arquivo das camadas

Agora vamos gerar um arquivo GeoPackage com um padrão de nomes assim:

* Tema - Geometria - Serviço

Isso tanto para os pontos quanto para as totalizações por distrito

In [77]:
log.write("## Gerando camadas de polígonos\n")

32

In [78]:
poligonos = gdf[~gdf.Serviço.isin(gdf_servicos_pontos.index)]

In [79]:
# gdf_servicos_pontos[gdf_servicos_pontos.index.str.contains('morcego')]

In [80]:
poligonos[['AGUARDANDO ATENDIMENTO','CANCELADA','FINALIZADA','INDEFERIDO']] = pd.get_dummies(poligonos['Status da solicitação'])

/home/fernando/miniconda3/envs/geopandas/lib/python3.8/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [81]:
poligonos.dtypes

index                              int64
Data de abertura          datetime64[ns]
Canal                             object
Tema                              object
Assunto                           object
Serviço                           object
Logradouro                        object
Número                           float64
CEP                               object
Subprefeitura                     object
Distrito                          object
Setor                            float64
Quadra                           float64
Latitude                         float64
Longitude                        float64
Data do parecer           datetime64[ns]
Status da solicitação             object
Órgão                             object
geometry                        geometry
AGUARDANDO ATENDIMENTO             uint8
CANCELADA                          uint8
FINALIZADA                         uint8
INDEFERIDO                         uint8
dtype: object

In [82]:
(poligonos[['Distrito', 'AGUARDANDO ATENDIMENTO', 'CANCELADA', 'FINALIZADA', 'INDEFERIDO', 'Data de abertura']]
 .groupby(['Distrito'])
 .agg({
     'AGUARDANDO ATENDIMENTO':'sum',
     'CANCELADA':'sum',
     'FINALIZADA':'sum',
     'INDEFERIDO':'sum',
     'Data de abertura':'count'
 })
 .rename(columns={'Data de abertura':'total_atendimentos'}))

,AGUARDANDO ATENDIMENTO,CANCELADA,FINALIZADA,INDEFERIDO,total_atendimentos
Distrito,,,,,
AGUA RASA,706.0,37,2965.0,382.0,4090
ALTO DE PINHEIROS,418.0,4,1856.0,268.0,2546
ANHANGUERA,311.0,5,1275.0,114.0,1705
ARICANDUVA,525.0,13,2322.0,314.0,3174
ARTUR ALVIM,512.0,15,2453.0,212.0,3192
...,...,...,...,...,...
VILA MARIANA,1007.0,33,4083.0,578.0,5701
VILA MATILDE,965.0,15,3204.0,360.0,4544
VILA MEDEIROS,840.0,21,4159.0,346.0,5366


In [83]:
df_distritos = df_distritos.drop('centroide', axis=1)

In [84]:
for t in poligonos.Tema.unique():
    log.write(f"{t}: ({len(poligonos[(poligonos.Tema == t)])})\n")
    for s in poligonos[(poligonos.Tema == t)].Serviço.unique():
        log.write(f"  {s}: ({len(poligonos[(poligonos.Serviço == s)])})\n")
        c = (poligonos[poligonos.Serviço == s][['Distrito', 'AGUARDANDO ATENDIMENTO', 'CANCELADA', 'FINALIZADA', 'INDEFERIDO', 'Data de abertura']]
             .groupby(['Distrito'])
             .agg({
                 'AGUARDANDO ATENDIMENTO':'sum',
                 'CANCELADA':'sum',
                 'FINALIZADA':'sum',
                 'INDEFERIDO':'sum',
                 'Data de abertura':'count'
             })
             .rename(columns={'Data de abertura':'total_atendimentos'}))
        r = df_distritos.merge(c, left_on='ds_nome', right_on='Distrito')
        if len(r) > 1:
            r.to_file(f'resultados/Visualizador-156-2020.gpkg', layer=f'{t} - Polígono - {s}', driver='GPKG')

In [85]:
log.write("## Gerando camadas de pontos\n")

29

In [86]:
gdf_sp = gpd.sjoin(gdf, gpd.GeoDataFrame([{'municipio':'São Paulo'}], geometry=msp.geometry.buffer(1000)), how='left', op='within')

In [87]:
# gpd.GeoDataFrame([{'municipio':'São Paulo'}], geometry=msp.geometry.buffer(1000)).plot()

In [88]:
# len(gdf_sp)

In [89]:
pontos = gdf[gdf.Serviço.isin(gdf_servicos_pontos.index) & ~gdf_sp.municipio.isna()]

In [90]:
gdf[gdf.Serviço.isin(gdf_servicos_pontos.index) & ~gdf_sp.municipio.isna()]
# gdf[gdf.Serviço.isin(gdf_servicos_pontos.index)]

,index,Data de abertura,Canal,Tema,Assunto,Serviço,Logradouro,Número,CEP,Subprefeitura,Distrito,Setor,Quadra,Latitude,Longitude,Data do parecer,Status da solicitação,Órgão,geometry
1,1,2020-01-01 00:01:18,APLICATIVO,Rua e bairro,"Reparos em asfalto, pontes e viadutos",Tapa-buraco,Rua Doutor Paulo Furtado de Oliveira,401.0,2.67403e+06,CASA VERDE-CACHOEIRINHA,CACHOEIRINHA,108.0,89.0,-23.459489,-46.667533,2020-01-04 20:35:15,FINALIZADA,SMSUB,POINT (329675.999 7404626.542)
2,2,2020-01-01 00:04:13,APLICATIVO,Rua e bairro,"Reparos em asfalto, pontes e viadutos",Tapa-buraco,Rua Doutor Paulo Furtado de Oliveira,341.0,2.67403e+06,CASA VERDE-CACHOEIRINHA,CACHOEIRINHA,108.0,89.0,-23.459793,-46.668002,2020-01-04 20:36:10,FINALIZADA,SMSUB,POINT (329628.476 7404592.355)
3,3,2020-01-01 00:06:57,APLICATIVO,Rua e bairro,"Reparos em asfalto, pontes e viadutos",Tapa-buraco,Rua Doutor Paulo Furtado de Oliveira,250.0,2.67403e+06,CASA VERDE-CACHOEIRINHA,CACHOEIRINHA,108.0,81.0,-23.460395,-46.668632,2020-01-04 20:38:05,FINALIZADA,SMSUB,POINT (329564.918 7404524.956)
4,4,2020-01-01 00:08:05,CENTRAL TELEFÔNICA,Trânsito e Transporte,Reclamação ônibus,Ônibus - Reclamação de intervalo excessivo da ...,Rua Kotinda,1679.0,2.36501e+06,JACANA-TREMEMBE,TREMEMBE,NaN,NaN,-23.425205,-46.582186,2020-02-08 06:11:29,FINALIZADA,SPTRANS,POINT (338353.381 7408521.455)
5,5,2020-01-01 00:09:19,APLICATIVO,Rua e bairro,"Reparos em asfalto, pontes e viadutos",Tapa-buraco,Rua Doutor Paulo Furtado de Oliveira,150.0,2.67403e+06,CASA VERDE-CACHOEIRINHA,CACHOEIRINHA,108.0,81.0,-23.460918,-46.669338,2020-01-04 20:36:12,FINALIZADA,SMSUB,POINT (329493.475 7404466.214)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
664923,664923,2020-06-30 23:03:21,PORTAL ONLINE,Rua e bairro,"Reparos em asfalto, pontes e viadutos",Tapa-buraco,Rua João Alves Pimenta,530.0,2.967e+06,FREGUESIA-BRASILANDIA,FREGUESIA DO O,307.0,71.0,-23.481408,-46.708198,2020-07-13 12:40:04,INDEFERIDO,SMSUB,POINT (325550.459 7402150.515)
664938,664938,2020-06-30 23:20:16,PORTAL ONLINE,Lixo e limpeza,Coleta seletiva,Instalação de contêiner para coleta seletiva,Rua Itapiru,405.0,4.14301e+06,VILA MARIANA,SAUDE,47.0,35.0,-23.615570,-46.635371,2020-07-02 17:30:33,FINALIZADA,AMLURB,POINT (333158.805 7387380.134)
664942,664942,2020-06-30 23:22:10,PORTAL ONLINE,Rua e bairro,"Reparos em asfalto, pontes e viadutos",Tapa-buraco,Rua dos Jequitibás,179.0,5.26303e+06,JABAQUARA,JABAQUARA,91.0,17.0,-23.646634,-46.642266,2020-07-07 23:13:27,FINALIZADA,SMSUB,POINT (332494.788 7383932.063)
664969,664969,2020-06-30 23:46:19,APLICATIVO,Lixo e limpeza,Lixeiras Públicas,Instalação e remoção de lixeira,Rua Adília Mercado Madureira,39.0,4.40107e+06,CIDADE ADEMAR,CIDADE ADEMAR,120.0,151.0,-23.666484,-46.665565,2020-07-01 17:51:01,FINALIZADA,AMLURB,POINT (330143.547 7381706.295)


In [91]:
pontos[pontos.Serviço.str.contains('Morcego')]

,index,Data de abertura,Canal,Tema,Assunto,Serviço,Logradouro,Número,CEP,Subprefeitura,Distrito,Setor,Quadra,Latitude,Longitude,Data do parecer,Status da solicitação,Órgão,geometry


In [92]:
gdf_servicos_pontos.index

Index(['(antigo) Reformar galerias de águas de chuvas (pluviais)',
       'Avaliação de animal em sofrimento sem proprietário para eutanásia (morte sem dor)',
       'Avisar sobre animal agressor em vias públicas',
       'Capinação em guias e sarjetas',
       'Cata Bagulho - Reclamação de não retirada de material',
       'Conserto de lixeira',
       'Denúncia de má conduta dos funcionários (motoristas /coletores)',
       'Denúncia de má conduta dos funcionários de limpeza',
       'Fiscalização de veículo estacionado em local proibido',
       'Implantação de coleta porta-porta',
       'Instalação de contêiner para coleta de lixo domiciliar',
       'Instalação de contêiner para coleta seletiva',
       'Instalação e remoção de lixeira',
       'Lavagem especial de locais públicos',
       'Limpeza da via pública após enchentes ou eventos',
       'Limpeza de bueiros, boca de lobo e poços de visita',
       'Manutenção de placas de trânsito',
       'Morcego - Reclamar sobre loca

In [93]:
pontos.Serviço.unique()

array(['Tapa-buraco',
       'Ônibus - Reclamação de intervalo excessivo da linha',
       'Fiscalização de veículo estacionado em local proibido',
       'Instalação e remoção de lixeira',
       'Reclamação de falta de varrição', 'Capinação em guias e sarjetas',
       'Ônibus - Denúncia de direção inadequada ou perigosa de veículo',
       'Limpeza de bueiros, boca de lobo e poços de visita',
       'Lavagem especial de locais públicos',
       'Cata Bagulho - Reclamação de não retirada de material',
       'Pintura de guias e postes', 'Manutenção de placas de trânsito',
       'Sinalização horizontal apagada (pintura de solo) \x96 Solicitar repintura',
       'Limpeza da via pública após enchentes ou eventos',
       'Reclamação de não remoção dos sacos de varrição',
       'Instalação de contêiner para coleta seletiva',
       'Instalação de contêiner para coleta de lixo domiciliar',
       'Conserto de lixeira',
       'Tapa-buraco em faixa exclusiva ou corredor de ônibus',
     

In [94]:
len(pontos)

103788

In [95]:
# gdf[gdf.Serviço.isin(gdf_servicos_pontos.index) & gdf_sp.municipio.isna()].plot()

In [96]:
for t in pontos.Tema.unique():
    log.write(f"{t}: ({len(pontos[(pontos.Tema == t)])})\n")
    for s in pontos[(pontos.Tema == t)].Serviço.unique():
        log.write(f"  {s}: ({len(pontos[(pontos.Serviço == s) & (pontos.Tema == t)])})\n")
        r = pontos[(pontos.Serviço == s) & (pontos.Tema == t)]
        if len(r) > 1:
            r.to_file(f'resultados/Visualizador-156-2020.gpkg', layer=f'{t} - Ponto - {s}', driver='GPKG')

In [97]:
pontos_menor_200 = gdf_menor_200[gdf_menor_200.geometry.is_valid]

In [98]:
len(pontos_menor_200)

598

In [99]:
log.write(f"Outros: ({len(pontos_menor_200)})\n")
pontos_menor_200.to_file(f'resultados/Visualizador-156-2020.gpkg', layer=f'Outros - Serviços com menos de {limite} ocorrências - Ponto', driver='GPKG')

## Salvando LOG


In [100]:
log.write(f'- {datetime.now().strftime("%d/%m/%Y %H:%M:%S")}')
log.write('## Finalizando processamento')

28

In [101]:
log.close()

In [102]:
# gdf